In [1]:
import tensorflow as tf
import keras
import gensim
import sklearn
import nltk
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='NanumBarunGothic')

In [2]:
df_care_only = pd.read_csv('./df_care_only.csv', encoding='utf-8')
df_care_only

,Unnamed: 0,title,reviewer,review_date,content,rate_int
0,0,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",n Butter Bread,2023년 8월 5일,네트워크오류라고 나와서 와이파이 끄고 하면 로그인 안되고 일시적인 오류라고 계속 뜹...,1
1,1,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",XXX XXX,2023년 10월 3일,카카오로 시작하기로 가입햇는데 로그인후 카카오로 로그인이 안됩니다 계속 튕기고 삭제...,1
2,2,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",신미경,2023년 9월 27일,앱열기가 안되네여.. 삭제후 재설치..권한도 설정들어가서 직접했는데도 앱자체가 안열...,1
3,3,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",백승희,2023년 10월 3일,카카오로 시작하기가 안됩니다. 크롬에서 카카오로 가입 후 건뱡신청 했는데 어플에서는...,4
4,4,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",Jeju life Jaymi's,2023년 9월 28일,앱 접속할때마다 네트워크가 없다고 떠요. 어떻게 사용하라는건가요? 그리고 간병금액 ...,2
...,...,...,...,...,...,...
2422,2422,"케어네이션 - 일감 찾기, 간병인, 동행인, 요양보호사",추유린,2021년 9월 6일,앱통장 편해~,4
2423,2423,"케어네이션 - 일감 찾기, 간병인, 동행인, 요양보호사",장주미,2021년 8월 16일,좋습니다,5
2424,2424,"케어네이션 - 일감 찾기, 간병인, 동행인, 요양보호사",한옥분,2020년 10월 19일,좋아요,5
2425,2425,"케어네이션 - 일감 찾기, 간병인, 동행인, 요양보호사",해피스마일,2021년 8월 30일,Good!!😙😙,5


In [3]:
rate_int = [0 if rate_int in range(0,5) else 1 for rate_int in df_care_only['rate_int']]
pos_cnt = len([x for x in rate_int if x==1])
neg_cnt = len([x for x in rate_int if x==0])
print(("긍정비율 : {:.4f}%,".format(pos_cnt/len(rate_int))+" 긍정개수 : {}개".format(pos_cnt)))
print(("부정비율 : {:.4f}%,".format(neg_cnt/len(rate_int))+" 부정개수 : {}개".format(neg_cnt)))

긍정비율 : 0.7478%, 긍정개수 : 1815개
부정비율 : 0.2522%, 부정개수 : 612개


In [4]:
from pykospacing import Spacing
spacing=Spacing()

In [5]:
import kss
from konlpy.tag import Okt
from konlpy.tag import Kkma
import re

okt = Okt()
kkma = Kkma()

In [6]:
replace_list = pd.read_excel('./replace_list.xlsx')
replace_list[-10:]

,before_replacement,after_replacement
373,안되염,안됩니다
374,않됨여,안됩니다
375,않돼여,안됩니다
376,않됌여,안됩니다
377,👍,굿
378,♥,좋습니다
379,♡,좋습니다
380,😄,좋습니다
381,Good,굿
382,good,굿


In [7]:
df_care_only['spaced_review'] = df_care_only['content'].apply(spacing)
df_care_only[:3]

,Unnamed: 0,title,reviewer,review_date,content,rate_int,spaced_review
0,0,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",n Butter Bread,2023년 8월 5일,네트워크오류라고 나와서 와이파이 끄고 하면 로그인 안되고 일시적인 오류라고 계속 뜹...,1,네트워크 오류라고 나와서 와이파이 끄고 하면 로그인 안 되고 일시적인 오류라고 계속...
1,1,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",XXX XXX,2023년 10월 3일,카카오로 시작하기로 가입햇는데 로그인후 카카오로 로그인이 안됩니다 계속 튕기고 삭제...,1,카카오로 시작하기로 가입 햇는데 로그인 후 카카오로 로그인이 안 됩니다 계속 튕기고...
2,2,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",신미경,2023년 9월 27일,앱열기가 안되네여.. 삭제후 재설치..권한도 설정들어가서 직접했는데도 앱자체가 안열...,1,앱 열기가 안 되네 여.. 삭제 후 재 설치..권한도 설정 들어가서 직접 했는데도 ...


In [16]:
from konlpy.tag import Okt

In [10]:
def tokenize_reviews_list(reviews):
    tokens = okt.morphs(reviews)
    return ' '.join(tokens)

In [11]:
df_care_only['tokenized_review'] = df_care_only['spaced_review'].apply(tokenize_reviews_list)
df_care_only[:2]

,Unnamed: 0,title,reviewer,review_date,content,rate_int,spaced_review,tokenized_review
0,0,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",n Butter Bread,2023년 8월 5일,네트워크오류라고 나와서 와이파이 끄고 하면 로그인 안되고 일시적인 오류라고 계속 뜹...,1,네트워크 오류라고 나와서 와이파이 끄고 하면 로그인 안 되고 일시적인 오류라고 계속...,네트워크 오류 라고 나와서 와이파이 끄고 하면 로그인 안 되고 일시 적 인 오류 라...
1,1,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",XXX XXX,2023년 10월 3일,카카오로 시작하기로 가입햇는데 로그인후 카카오로 로그인이 안됩니다 계속 튕기고 삭제...,1,카카오로 시작하기로 가입 햇는데 로그인 후 카카오로 로그인이 안 됩니다 계속 튕기고...,카카오 로 시작 하기로 가입 햇 는데 로그인 후 카카오 로 로그인 이 안 됩니다 계...


In [12]:
def replace_word(tokenized_review):
    replaced_review = tokenized_review
    for i in range(len(replace_list['before_replacement'])):
        try:
            if replace_list['before_replacement'][i] in replaced_review:
                replaced_review = replaced_review.replace(replace_list['before_replacement'][i], replace_list['after_replacement'][i])
        except Exception as e:
            # print(f"Error 발생 / 에러명: {e}")
            pass
    return replaced_review

In [13]:
df_care_only['replaced_review'] = df_care_only['tokenized_review'].apply(replace_word)
df_care_only[:5]

,Unnamed: 0,title,reviewer,review_date,content,rate_int,spaced_review,tokenized_review,replaced_review
0,0,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",n Butter Bread,2023년 8월 5일,네트워크오류라고 나와서 와이파이 끄고 하면 로그인 안되고 일시적인 오류라고 계속 뜹...,1,네트워크 오류라고 나와서 와이파이 끄고 하면 로그인 안 되고 일시적인 오류라고 계속...,네트워크 오류 라고 나와서 와이파이 끄고 하면 로그인 안 되고 일시 적 인 오류 라...,네트워크 오류 라고 나와서 와이파이 끄고 하면 로그인 안 되고 일시 적 인 오류 라...
1,1,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",XXX XXX,2023년 10월 3일,카카오로 시작하기로 가입햇는데 로그인후 카카오로 로그인이 안됩니다 계속 튕기고 삭제...,1,카카오로 시작하기로 가입 햇는데 로그인 후 카카오로 로그인이 안 됩니다 계속 튕기고...,카카오 로 시작 하기로 가입 햇 는데 로그인 후 카카오 로 로그인 이 안 됩니다 계...,카카오톡 로 시작 하기로 회원가입 햇 는데 로그인 후 카카오톡 로 로그인 이 안됩니...
2,2,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",신미경,2023년 9월 27일,앱열기가 안되네여.. 삭제후 재설치..권한도 설정들어가서 직접했는데도 앱자체가 안열...,1,앱 열기가 안 되네 여.. 삭제 후 재 설치..권한도 설정 들어가서 직접 했는데도 ...,앱 열기 가 안 되네 여 .. 삭제 후 재 설치 .. 권한 도 설정 들어가서 직접 ...,애플리케이션 열기 가 안 되네 여 .. 삭제 후 재 설치 .. 권한 도 설정 들어가...
3,3,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",백승희,2023년 10월 3일,카카오로 시작하기가 안됩니다. 크롬에서 카카오로 가입 후 건뱡신청 했는데 어플에서는...,4,카카오로 시작하기가 안 됩니다. 크롬에서 카카오로 가입 후 건 뱡 신청 했는데 어플...,카카오 로 시작 하기가 안 됩니다 . 크롬 에서 카카오 로 가입 후 건 뱡 신청 했...,카카오톡 로 시작 하기가 안됩니다 . 크롬 에서 카카오톡 로 회원가입 후 건 뱡 신...
4,4,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",Jeju life Jaymi's,2023년 9월 28일,앱 접속할때마다 네트워크가 없다고 떠요. 어떻게 사용하라는건가요? 그리고 간병금액 ...,2,앱 접속할 때마다 네트워크가 없다고 떠요. 어떻게 사용하라는 건가요? 그리고 간병 ...,앱 접속 할 때 마다 네트워크 가 없다고 떠요 . 어떻게 사용 하 라는 건가 요 ?...,애플리케이션 접속 할 때 마다 네트워크 가 없다고 떠요 . 어떻게 사용 하 라는 건...


In [15]:
with open('./stopwords.txt', 'r', encoding = 'utf-8') as f:
    list_file = f.readlines()
stopwords = list_file[0].split(",")


# 토크나이징 함수 정의(한자 이상 적용. 불용어 삭제)
def stopword(raw, pos=["Noun",'Adjective',"Alpha","Verb","Number"], stopword=stopwords):
    result_list = list()
    # for word in replaced_review :
    for word, tag in okt.pos(raw, norm=True, stem=True) :
        if tag in pos and word not in stopword:
            result_list.append(word)
    # 일부 경우(토큰 갯수 제한)
    # if len(result_list) <= 3 or len(result_list) < 12 :
    #   result_list
    return ' '.join(result_list) # return str

In [16]:
df_care_only['stopword_review'] = df_care_only['replaced_review'].apply(stopword)
df_care_only[:5]

,Unnamed: 0,title,reviewer,review_date,content,rate_int,spaced_review,tokenized_review,replaced_review,stopword_review
0,0,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",n Butter Bread,2023년 8월 5일,네트워크오류라고 나와서 와이파이 끄고 하면 로그인 안되고 일시적인 오류라고 계속 뜹...,1,네트워크 오류라고 나와서 와이파이 끄고 하면 로그인 안 되고 일시적인 오류라고 계속...,네트워크 오류 라고 나와서 와이파이 끄고 하면 로그인 안 되고 일시 적 인 오류 라...,네트워크 오류 라고 나와서 와이파이 끄고 하면 로그인 안 되고 일시 적 인 오류 라...,네트워크 오류 와이파이 로그인 안 일시 적 인 오류 뜨다 간병인 결정 결제 주말 이...
1,1,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",XXX XXX,2023년 10월 3일,카카오로 시작하기로 가입햇는데 로그인후 카카오로 로그인이 안됩니다 계속 튕기고 삭제...,1,카카오로 시작하기로 가입 햇는데 로그인 후 카카오로 로그인이 안 됩니다 계속 튕기고...,카카오 로 시작 하기로 가입 햇 는데 로그인 후 카카오 로 로그인 이 안 됩니다 계...,카카오톡 로 시작 하기로 회원가입 햇 는데 로그인 후 카카오톡 로 로그인 이 안됩니...,카카오 톡 시작 회원 가입 햇 로그인 후 카카오 톡 로그인 튕기 고 삭제 후 재 설...
2,2,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",신미경,2023년 9월 27일,앱열기가 안되네여.. 삭제후 재설치..권한도 설정들어가서 직접했는데도 앱자체가 안열...,1,앱 열기가 안 되네 여.. 삭제 후 재 설치..권한도 설정 들어가서 직접 했는데도 ...,앱 열기 가 안 되네 여 .. 삭제 후 재 설치 .. 권한 도 설정 들어가서 직접 ...,애플리케이션 열기 가 안 되네 여 .. 삭제 후 재 설치 .. 권한 도 설정 들어가...,애플리케이션 열기 안 삭제 후 재 설치 권한 도 설정 들어가다 직접 애플리케이션 안...
3,3,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",백승희,2023년 10월 3일,카카오로 시작하기가 안됩니다. 크롬에서 카카오로 가입 후 건뱡신청 했는데 어플에서는...,4,카카오로 시작하기가 안 됩니다. 크롬에서 카카오로 가입 후 건 뱡 신청 했는데 어플...,카카오 로 시작 하기가 안 됩니다 . 크롬 에서 카카오 로 가입 후 건 뱡 신청 했...,카카오톡 로 시작 하기가 안됩니다 . 크롬 에서 카카오톡 로 회원가입 후 건 뱡 신...,카카오 톡 시작 크롬 카카오 톡 회원 가입 후 건 뱡 신청 어플 카카오 톡 시작 아...
4,4,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",Jeju life Jaymi's,2023년 9월 28일,앱 접속할때마다 네트워크가 없다고 떠요. 어떻게 사용하라는건가요? 그리고 간병금액 ...,2,앱 접속할 때마다 네트워크가 없다고 떠요. 어떻게 사용하라는 건가요? 그리고 간병 ...,앱 접속 할 때 마다 네트워크 가 없다고 떠요 . 어떻게 사용 하 라는 건가 요 ?...,애플리케이션 접속 할 때 마다 네트워크 가 없다고 떠요 . 어떻게 사용 하 라는 건...,애플리케이션 접속 네트워크 뜨다 사용 건가 간병 금액 상한선 건 하루 당 30만원 ...


In [19]:
def remove_one_word(stopword_review):
    word_list = []
    words = stopword_review.split()
    for x in words:
        if len(x)>1 or (x in ["굿", "굳", "짱", "돈", "값"]):
            word_list.append(x)
    return ' '.join(word_list)

df_care_only['one_word_removed'] = df_care_only['stopword_review'].apply(remove_one_word)
df_care_only[-10:]

,Unnamed: 0,title,reviewer,review_date,content,rate_int,spaced_review,tokenized_review,replaced_review,stopword_review,one_word_removed
2417,2417,"케어네이션 - 일감 찾기, 간병인, 동행인, 요양보호사",양동우,2021년 12월 6일,강추합니다,5,강추합니다,강 추합니다,강 추합니다,강 추하다,추하다
2418,2418,"케어네이션 - 일감 찾기, 간병인, 동행인, 요양보호사",강현준,2021년 7월 6일,굿잉요,5,굿잉요,굿잉 요,굿잉 요,굿잉,굿잉
2419,2419,"케어네이션 - 일감 찾기, 간병인, 동행인, 요양보호사",배연희,2021년 11월 16일,대체로 만족,3,대체로 만족,대체로 만족,대체로 만족,대체로 만족,대체로 만족
2420,2420,"케어네이션 - 일감 찾기, 간병인, 동행인, 요양보호사",silky,2021년 11월 16일,최고예요,5,최고예요,최고 예요,최고 예요,최고,최고
2421,2421,"케어네이션 - 일감 찾기, 간병인, 동행인, 요양보호사",유승하,2021년 6월 20일,깔끔하네요,5,깔끔하네요,깔끔하네요,깔끔하네요,깔끔하다,깔끔하다
2422,2422,"케어네이션 - 일감 찾기, 간병인, 동행인, 요양보호사",추유린,2021년 9월 6일,앱통장 편해~,4,앱 통장 편해~,앱 통장 편해 ~,애플리케이션 통장 편해 ~,애플리케이션 통장 편하다,애플리케이션 통장 편하다
2423,2423,"케어네이션 - 일감 찾기, 간병인, 동행인, 요양보호사",장주미,2021년 8월 16일,좋습니다,5,좋습니다,좋습니다,좋습니다,좋다,좋다
2424,2424,"케어네이션 - 일감 찾기, 간병인, 동행인, 요양보호사",한옥분,2020년 10월 19일,좋아요,5,좋아요,좋아요,좋아요,좋다,좋다
2425,2425,"케어네이션 - 일감 찾기, 간병인, 동행인, 요양보호사",해피스마일,2021년 8월 30일,Good!!😙😙,5,Good !!😙😙,Good !! 😙😙,굿 !! 😙😙,굿,굿
2426,2426,"케어네이션 - 일감 찾기, 간병인, 동행인, 요양보호사",샛노란해바라기,2021년 8월 21일,Good~~^^,5,Good~~,Good ~~,굿 ~~,굿,굿


In [20]:
df_care_only['one_word_removed'].isnull().sum()

0

In [22]:
df_care_only.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2427 entries, 0 to 2426
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        2427 non-null   int64 
 1   title             2427 non-null   object
 2   reviewer          2427 non-null   object
 3   review_date       2427 non-null   object
 4   content           2427 non-null   object
 5   rate_int          2427 non-null   int64 
 6   spaced_review     2427 non-null   object
 7   tokenized_review  2427 non-null   object
 8   replaced_review   2427 non-null   object
 9   stopword_review   2427 non-null   object
 10  one_word_removed  2427 non-null   object
dtypes: int64(2), object(9)
memory usage: 208.7+ KB


In [23]:
import numpy as np

df_care_only['classification'] = np.where(df_care_only['rate_int'] >= 5, '긍정', '부정')
df_care_only[:5]

,Unnamed: 0,title,reviewer,review_date,content,rate_int,spaced_review,tokenized_review,replaced_review,stopword_review,one_word_removed,classification
0,0,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",n Butter Bread,2023년 8월 5일,네트워크오류라고 나와서 와이파이 끄고 하면 로그인 안되고 일시적인 오류라고 계속 뜹...,1,네트워크 오류라고 나와서 와이파이 끄고 하면 로그인 안 되고 일시적인 오류라고 계속...,네트워크 오류 라고 나와서 와이파이 끄고 하면 로그인 안 되고 일시 적 인 오류 라...,네트워크 오류 라고 나와서 와이파이 끄고 하면 로그인 안 되고 일시 적 인 오류 라...,네트워크 오류 와이파이 로그인 안 일시 적 인 오류 뜨다 간병인 결정 결제 주말 이...,네트워크 오류 와이파이 로그인 일시 오류 뜨다 간병인 결정 결제 주말 이르다 고객 ...,부정
1,1,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",XXX XXX,2023년 10월 3일,카카오로 시작하기로 가입햇는데 로그인후 카카오로 로그인이 안됩니다 계속 튕기고 삭제...,1,카카오로 시작하기로 가입 햇는데 로그인 후 카카오로 로그인이 안 됩니다 계속 튕기고...,카카오 로 시작 하기로 가입 햇 는데 로그인 후 카카오 로 로그인 이 안 됩니다 계...,카카오톡 로 시작 하기로 회원가입 햇 는데 로그인 후 카카오톡 로 로그인 이 안됩니...,카카오 톡 시작 회원 가입 햇 로그인 후 카카오 톡 로그인 튕기 고 삭제 후 재 설...,카카오 시작 회원 가입 로그인 카카오 로그인 튕기 삭제 설치 해도 튕깁 로그인 회원...,부정
2,2,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",신미경,2023년 9월 27일,앱열기가 안되네여.. 삭제후 재설치..권한도 설정들어가서 직접했는데도 앱자체가 안열...,1,앱 열기가 안 되네 여.. 삭제 후 재 설치..권한도 설정 들어가서 직접 했는데도 ...,앱 열기 가 안 되네 여 .. 삭제 후 재 설치 .. 권한 도 설정 들어가서 직접 ...,애플리케이션 열기 가 안 되네 여 .. 삭제 후 재 설치 .. 권한 도 설정 들어가...,애플리케이션 열기 안 삭제 후 재 설치 권한 도 설정 들어가다 직접 애플리케이션 안...,애플리케이션 열기 삭제 설치 권한 설정 들어가다 직접 애플리케이션 열리다,부정
3,3,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",백승희,2023년 10월 3일,카카오로 시작하기가 안됩니다. 크롬에서 카카오로 가입 후 건뱡신청 했는데 어플에서는...,4,카카오로 시작하기가 안 됩니다. 크롬에서 카카오로 가입 후 건 뱡 신청 했는데 어플...,카카오 로 시작 하기가 안 됩니다 . 크롬 에서 카카오 로 가입 후 건 뱡 신청 했...,카카오톡 로 시작 하기가 안됩니다 . 크롬 에서 카카오톡 로 회원가입 후 건 뱡 신...,카카오 톡 시작 크롬 카카오 톡 회원 가입 후 건 뱡 신청 어플 카카오 톡 시작 아...,카카오 시작 크롬 카카오 회원 가입 신청 어플 카카오 시작 아예 난처하다 간병 서비...,부정
4,4,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",Jeju life Jaymi's,2023년 9월 28일,앱 접속할때마다 네트워크가 없다고 떠요. 어떻게 사용하라는건가요? 그리고 간병금액 ...,2,앱 접속할 때마다 네트워크가 없다고 떠요. 어떻게 사용하라는 건가요? 그리고 간병 ...,앱 접속 할 때 마다 네트워크 가 없다고 떠요 . 어떻게 사용 하 라는 건가 요 ?...,애플리케이션 접속 할 때 마다 네트워크 가 없다고 떠요 . 어떻게 사용 하 라는 건...,애플리케이션 접속 네트워크 뜨다 사용 건가 간병 금액 상한선 건 하루 당 30만원 ...,애플리케이션 접속 네트워크 뜨다 사용 건가 간병 금액 상한선 하루 30만원 지원 간...,부정


In [24]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [25]:
reviews_list = list(df_care_only['content'])
reviews_list[:5]

['네트워크오류라고 나와서 와이파이 끄고 하면 로그인 안되고 일시적인 오류라고 계속 뜹니다. 간병인 결정하고 결제해야 하는데 주말이라 고객센터도 전화안되고 어떡하죠? 빨리 해결되기를 바랍니다.',
 '카카오로 시작하기로 가입햇는데 로그인후 카카오로 로그인이 안됩니다 계속 튕기고 삭제후 재설치 해도 계속 튕깁니다 왜이런가요 아니 이거 로그인이 안되니 회원 탈퇴도 안되고 재가입도 안되고 뭘 할수있는게 아무것도 없네',
 '앱열기가 안되네여.. 삭제후 재설치..권한도 설정들어가서 직접했는데도 앱자체가 안열려요...',
 '카카오로 시작하기가 안됩니다. 크롬에서 카카오로 가입 후 건뱡신청 했는데 어플에서는 카카오로 시작하기가 아예 안돼서 난처하네요. 간병서비스 신청하는 것 자체는 유용합니다.',
 '앱 접속할때마다 네트워크가 없다고 떠요. 어떻게 사용하라는건가요? 그리고 간병금액 상한선 같은건 없나요? 하루 일당 30만원도 넘게 지원하는 간병인도 있는데 그냥 되는대로 막 부르는건지요..']

In [26]:
countVectorizer = CountVectorizer()
countVectorizer.fit(reviews_list)

CountVectorizer()

In [27]:
countVectorizer.vocabulary_

{'네트워크오류라고': 1678,
 '나와서': 1520,
 '와이파이': 5465,
 '끄고': 1466,
 '하면': 7962,
 '로그인': 2411,
 '안되고': 4691,
 '일시적인': 5934,
 '오류라고': 5403,
 '계속': 775,
 '뜹니다': 2399,
 '간병인': 307,
 '결정하고': 720,
 '결제해야': 740,
 '하는데': 7927,
 '주말이라': 6872,
 '고객센터도': 793,
 '전화안되고': 6563,
 '어떡하죠': 5000,
 '빨리': 3633,
 '해결되기를': 8087,
 '바랍니다': 3046,
 '카카오로': 7493,
 '시작하기로': 4296,
 '가입햇는데': 180,
 '로그인후': 2414,
 '로그인이': 2412,
 '안됩니다': 4715,
 '튕기고': 7683,
 '삭제후': 3774,
 '재설치': 6427,
 '해도': 8102,
 '튕깁니다': 7685,
 '왜이런가요': 5487,
 '아니': 4556,
 '이거': 5674,
 '안되니': 4699,
 '회원': 8373,
 '탈퇴도': 7632,
 '재가입도': 6412,
 '할수있는게': 8056,
 '아무것도': 4586,
 '없네': 5196,
 '앱열기가': 4947,
 '안되네여': 4694,
 '권한도': 1207,
 '설정들어가서': 4039,
 '직접했는데도': 7070,
 '앱자체가': 4970,
 '안열려요': 4760,
 '시작하기가': 4295,
 '크롬에서': 7608,
 '가입': 164,
 '건뱡신청': 661,
 '했는데': 8173,
 '어플에서는': 5110,
 '아예': 4623,
 '안돼서': 4687,
 '난처하네요': 1545,
 '간병서비스': 270,
 '신청하는': 4361,
 '자체는': 6275,
 '유용합니다': 5627,
 '접속할때마다': 6603,
 '네트워크가': 1677,
 '없다고': 5205,
 '떠요': 2377,
 '어떻게': 5008,
 '사용하라는

In [28]:
tfidfVectorizer = TfidfVectorizer(use_idf=True)
tfidfVectorizer.fit(reviews_list) # 2차원 배열로 교육

TfidfVectorizer()

In [29]:
tfidfVectorizer.idf_

array([8.10167597, 8.10167597, 8.10167597, ..., 8.10167597, 8.10167597,
       6.71538161])

In [30]:
features = tfidfVectorizer.transform(reviews_list) # 학습 시 2차원 이용했으므로 변환도 2차원 적용
features

<2427x8461 sparse matrix of type '<class 'numpy.float64'>'
	with 22063 stored elements in Compressed Sparse Row format>

In [32]:
target = list(df_care_only['classification'])
target[:10]

['부정', '부정', '부정', '부정', '부정', '부정', '부정', '부정', '긍정', '긍정']

In [33]:
from sklearn.svm import SVC

In [34]:
model = SVC()
model.fit(features, target)

SVC()

In [36]:
df_for_ml = df_care_only['one_word_removed']

In [37]:
features_review = tfidfVectorizer.transform(df_for_ml)
features_review

<2427x8461 sparse matrix of type '<class 'numpy.float64'>'
	with 7815 stored elements in Compressed Sparse Row format>

In [38]:
model.predict(features_review)

array(['긍정', '긍정', '긍정', ..., '긍정', '긍정', '긍정'], dtype='<U2')

In [39]:
df_for_ml['classification'] = model.predict(features_review)
df_for_ml[:5]

C:\Users\moony\AppData\Local\Temp\ipykernel_6580\4274818501.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_ml['classification'] = model.predict(features_review)


0    네트워크 오류 와이파이 로그인 일시 오류 뜨다 간병인 결정 결제 주말 이르다 고객 ...
1    카카오 시작 회원 가입 로그인 카카오 로그인 튕기 삭제 설치 해도 튕깁 로그인 회원...
2             애플리케이션 열기 삭제 설치 권한 설정 들어가다 직접 애플리케이션 열리다
3    카카오 시작 크롬 카카오 회원 가입 신청 어플 카카오 시작 아예 난처하다 간병 서비...
4    애플리케이션 접속 네트워크 뜨다 사용 건가 간병 금액 상한선 하루 30만원 지원 간...
Name: one_word_removed, dtype: object

In [42]:
total_data = df_care_only[["title","reviewer","review_date","content", "one_word_removed", "rate_int", "classification"]]
total_data

,title,reviewer,review_date,content,one_word_removed,rate_int,classification
0,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",n Butter Bread,2023년 8월 5일,네트워크오류라고 나와서 와이파이 끄고 하면 로그인 안되고 일시적인 오류라고 계속 뜹...,네트워크 오류 와이파이 로그인 일시 오류 뜨다 간병인 결정 결제 주말 이르다 고객 ...,1,부정
1,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",XXX XXX,2023년 10월 3일,카카오로 시작하기로 가입햇는데 로그인후 카카오로 로그인이 안됩니다 계속 튕기고 삭제...,카카오 시작 회원 가입 로그인 카카오 로그인 튕기 삭제 설치 해도 튕깁 로그인 회원...,1,부정
2,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",신미경,2023년 9월 27일,앱열기가 안되네여.. 삭제후 재설치..권한도 설정들어가서 직접했는데도 앱자체가 안열...,애플리케이션 열기 삭제 설치 권한 설정 들어가다 직접 애플리케이션 열리다,1,부정
3,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",백승희,2023년 10월 3일,카카오로 시작하기가 안됩니다. 크롬에서 카카오로 가입 후 건뱡신청 했는데 어플에서는...,카카오 시작 크롬 카카오 회원 가입 신청 어플 카카오 시작 아예 난처하다 간병 서비...,4,부정
4,"케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기",Jeju life Jaymi's,2023년 9월 28일,앱 접속할때마다 네트워크가 없다고 떠요. 어떻게 사용하라는건가요? 그리고 간병금액 ...,애플리케이션 접속 네트워크 뜨다 사용 건가 간병 금액 상한선 하루 30만원 지원 간...,2,부정
...,...,...,...,...,...,...,...
2422,"케어네이션 - 일감 찾기, 간병인, 동행인, 요양보호사",추유린,2021년 9월 6일,앱통장 편해~,애플리케이션 통장 편하다,4,부정
2423,"케어네이션 - 일감 찾기, 간병인, 동행인, 요양보호사",장주미,2021년 8월 16일,좋습니다,좋다,5,긍정
2424,"케어네이션 - 일감 찾기, 간병인, 동행인, 요양보호사",한옥분,2020년 10월 19일,좋아요,좋다,5,긍정
2425,"케어네이션 - 일감 찾기, 간병인, 동행인, 요양보호사",해피스마일,2021년 8월 30일,Good!!😙😙,굿,5,긍정


In [45]:
total_data.to_csv('./total_data.csv')

In [47]:
df = total_data

In [48]:
# 긍정 / 부정 분리
positive_count = len(df[df['classification'] == '긍정'])
negative_count = len(df[df['classification'] == '부정'])

total_count = positive_count + negative_count

# 비율 계산
positive_ratio = (positive_count / total_count) * 100
negative_ratio = (negative_count / total_count) * 100

print(f"긍정 개수: {positive_count}, 비율: {positive_ratio:.2f}%")
print(f"부정 개수: {negative_count}, 비율: {negative_ratio:.2f}%")

긍정 개수: 1815, 비율: 74.78%
부정 개수: 612, 비율: 25.22%


In [49]:

NUM_TOPICS = 10
PASSES = 15

In [51]:
import gensim
from gensim import corpora
from gensim.models import LdaModel

In [52]:
# 2. LDA 모델링을 위한 함수 정의

def lda_modeling(review_prep):
    dictionary = corpora.Dictionary(review_prep)
    corpus = [dictionary.doc2bow(review) for review in review_prep]
    model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=PASSES)
    return model, corpus, dictionary

In [54]:
# 3. 긍정 리뷰 데이터 추출

review_prep_pos = df[df['classification'] == '긍정']['one_word_removed'].dropna().str.split().tolist()

In [55]:
# 4. LDA 모델링 수행

model, _, _ = lda_modeling(review_prep_pos)

In [56]:
# 5. 토픽별 주요 단어와 그 비율 출력
NUM_WORDS = 10

# 5. 토픽별 주요 단어와 그 비율 출력
topics_pos = model.print_topics(num_words=NUM_WORDS)

topics_output_pos = []
for topic in topics_pos:
    words_with_quotes = []
    for word_prob in topic[1].split(' + '):
        prob, word = word_prob.split('*')
        word_with_quote = f'{prob}*" {word.strip()} "'
        words_with_quotes.append(word_with_quote)
    topic_str = " + ".join(words_with_quotes)
    topics_output_pos.append(topic_str)

topics_output_pos

['0.087*" "좋다" " + 0.054*" "간병인" " + 0.052*" "리뷰" " + 0.040*" "확인" " + 0.034*" "선택" " + 0.021*" "애플리케이션" " + 0.018*" "일자리" " + 0.017*" "간병" " + 0.016*" "많다" " + 0.016*" "사용" "',
 '0.084*" "간병인" " + 0.032*" "많다" " + 0.027*" "간병" " + 0.024*" "좋다" " + 0.024*" "해주다" " + 0.021*" "자다" " + 0.020*" "매칭" " + 0.015*" "감사하다" " + 0.013*" "선택" " + 0.013*" "확인" "',
 '0.049*" "간병인" " + 0.026*" "선택" " + 0.025*" "간병" " + 0.024*" "많다" " + 0.021*" "경력" " + 0.018*" "지방" " + 0.015*" "힘들다" " + 0.015*" "좋다" " + 0.013*" "지원" " + 0.012*" "일감" "',
 '0.051*" "간병인" " + 0.049*" "좋다" " + 0.043*" "사용" " + 0.028*" "도움" " + 0.025*" "추천" " + 0.025*" "편하다" " + 0.023*" "간병" " + 0.022*" "필요하다" " + 0.022*" "애플리케이션" " + 0.016*" "자다" "',
 '0.062*" "좋다" " + 0.035*" "감사하다" " + 0.031*" "아주" " + 0.022*" "간병인" " + 0.022*" "자다" " + 0.020*" "엄마" " + 0.018*" "입원" " + 0.014*" "신경" " + 0.011*" "쓸다" " + 0.011*" "영상" "',
 '0.078*" "간병인" " + 0.062*" "케어" " + 0.051*" "네이션" " + 0.041*" "자다" " + 0.028*" "감사하다" " + 0.027*" "간병" " + 0.027*" "

In [57]:
!pip install pyLDAvis

     ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
     ------- -------------------------------- 0.5/2.6 MB 14.7 MB/s eta 0:00:01
     ------------------------- -------------- 1.7/2.6 MB 21.0 MB/s eta 0:00:01
     ---------------------------------------  2.6/2.6 MB 23.5 MB/s eta 0:00:01
     ---------------------------------------- 2.6/2.6 MB 18.4 MB/s eta 0:00:00
  Obtaining dependency information for pandas>=2.0.0 from https://files.pythonhosted.org/packages/2d/5e/9213ea10ac473e2437dc2cb17323ddc0999997e2713d6a0b683b10773994/pandas-2.1.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/341.8 kB ? eta -:--:--
     ------------------------------------- 341.8/341.8 kB 20.7 MB/s eta 0:00:00
   ----------------------------------------

DEPRECATION: Loading egg at c:\users\moony\anaconda3\lib\site-packages\argparse-1.4.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\moony\anaconda3\lib\site-packages\pykospacing-0.5-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\moony\anaconda3\lib\site-packages\py_hanspell-1.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 2.1.1 requires sentencepiece, which is not installed.
pandas-profiling 3.2.0 requires joblib~=1.1.0, but you have joblib 1.3.2 which is incompatibl

In [59]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
# Retrieve the corpus and dictionary using the lda_modeling function
model, corpus, dictionary = lda_modeling(review_prep_pos)

# Prepare the visualization data
vis_data = gensimvis.prepare(model, corpus, dictionary, n_jobs=-1)

# Display the visualization
pyLDAvis.display(vis_data)

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.


In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Prepare the visualization data
vis_data = gensimvis.prepare(model, corpus, dictionary)

# Display the visualization
pyLDAvis.display(vis_data)